# NYC taxi data regression

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription. [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace. [Check this notebook for creating a workspace](/sdk/resources/workspace/workspace.ipynb) 
- A Compute Cluster. [Check this notebook to create a compute cluster](/sdk/resources/compute/compute.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](/sdk/README.md#getting-started)

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Define different `commandComponent` using YAML
- Create `pipeline` load these components from YAML

**Motivations** - This notebook explains how to loadcomponent via SDK then use these components to build pipeline. We use NYC dataset, build pipeline with five steps, prep data, transform data, train model, predict results and evaluate model performance.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
#import required libraries
from azure.identity import InteractiveBrowserCredential
from azure.ml import MLClient, dsl
from azure.ml.entities import JobInput, load_component
from azure.ml._constants import AssetTypes

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [interactive authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.interactivebrowsercredential?view=azure-python) for this tutorial. More advanced connection methods can be found [here](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:

#Enter details of your AML workspace
subscription_id = '<SUBSCRIPTION_ID>'
resource_group = '<RESOURCE_GROUP>'
workspace = '<AML_WORKSPACE_NAME>'

In [ ]:
#get a handle to the workspace
ml_client = MLClient(InteractiveBrowserCredential(), subscription_id, resource_group, workspace)

# 2. Build pipeline

In [ ]:
parent_dir = ''

# 1. Load component funcs
prep_func = load_component(
    yaml_file=parent_dir + "./prep.yml"
)
transform_func = load_component(
    yaml_file=parent_dir + "./transform.yml"
)
train_func = load_component(
    yaml_file=parent_dir + "./train.yml"
)
predict_func = load_component(
    yaml_file=parent_dir + "./predict.yml"
)
score_func = load_component(
    yaml_file=parent_dir + "./score.yml"
)

# 2. Construct pipeline
@dsl.pipeline(default_compute="cpu-cluster", default_datastore="workspaceblobstore")
def sample_pipeline(pipeline_job_input):
    prep_job = prep_func(raw_data=pipeline_job_input)
    transform_job = transform_func(clean_data=prep_job.outputs.prep_data)
    train_job = train_func(training_data=transform_job.outputs.transformed_data)
    predict_job = predict_func(
        model_input=train_job.outputs.model_output, test_data=train_job.outputs.test_data
    )
    score_job = score_func(predictions=predict_job.outputs.predictions, model=train_job.outputs.model_output)
    return {
        "pipeline_job_prepped_data": prep_job.outputs.prep_data,
        "pipeline_job_transformed_data": transform_job.outputs.transformed_data,
        "pipeline_job_trained_model": train_job.outputs.model_output,
        "pipeline_job_test_data": train_job.outputs.test_data,
        "pipeline_job_predictions": predict_job.outputs.predictions,
        "pipeline_job_score_report": score_job.outputs.score_report,
    }

pipeline = sample_pipeline(
    JobInput(type=AssetTypes.URI_FOLDER, path=parent_dir + "./data/")
)
pipeline.outputs.pipeline_job_prepped_data.data = "/prepped_data"
pipeline.outputs.pipeline_job_prepped_data.mode = "rw_mount"
pipeline.outputs.pipeline_job_transformed_data.data = "/transformed_data"
pipeline.outputs.pipeline_job_transformed_data.mode = "rw_mount"
pipeline.outputs.pipeline_job_trained_model.data = "/trained-model"
pipeline.outputs.pipeline_job_trained_model.mode = "rw_mount"
pipeline.outputs.pipeline_job_test_data.data = "/test_data"
pipeline.outputs.pipeline_job_test_data.mode = "rw_mount"
pipeline.outputs.pipeline_job_predictions.data = "/predictions"
pipeline.outputs.pipeline_job_predictions.mode = "rw_mount"
pipeline.outputs.pipeline_job_score_report.data = "/report"
pipeline.outputs.pipeline_job_score_report.mode = "rw_mount"

## 3. Submit pipeline job

In [ ]:
# submit job to workspace
pipeline_job = ml_client.jobs.create_or_update(pipeline, experiment_name="nyc_taxi_data_regression", continue_run_on_step_failure=True)
pipeline_job.services["Studio"].endpoint

# Next Steps
You can see further examples of running a pipeline job [here](/sdk/jobs/pipelines/)